In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [2]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [3]:
english_stops = set(stopwords.words('english'))

In [4]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
17773    [you, gotta, fan, little, man, i, found, burle...
44056    [this, totally, odd, ball, feature, typical, p...
16060    [the, movie, good, story, line, action, good, ...
3399     [when, killer, calls, unusually, nasty, slashe...
28383    [i, suppose, like, pure, action, find, i, supp...
                               ...                        
49917    [long, trailer, whoever, said, got, joking, go...
4636     [the, directing, brilliant, casting, remarkabl...
32936    [i, watched, subtitled, russian, really, enjoy...
43004    [having, seen, loved, film, australia, i, keen...
37922    [yeah, i, watched, mini, series, my, mom, dad,...
Name: review, Length: 40000, dtype: object 

23165    [bronson, ireland, last, film, together, make,...
40666    [webs, starts, chicago, present, day, four, el...
14422    [valuable, king, tut, tomb, ok, maybe, that, v...
28936    [i, used, watch, show, i, little, girl, when, ...
30579    [not, like, i, went, expecting, lot, i, least,...
 

In [6]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [7]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[  104  3066   232 ...     0     0     0]
 [    8   364   935 ...   215 14054     1]
 [    2     3     9 ...     0     0     0]
 ...
 [    1   194  8302 ...     0     0     0]
 [ 1523    38   325 ...     0     0     0]
 [ 1033     1   194 ...     0     0     0]] 

Encoded X Test
 [[ 4314  3601   142 ...     0     0     0]
 [16356   450  3410 ...   484   626   825]
 [ 4620   610 36102 ...  2189    50   426]
 ...
 [ 2699  6199    24 ...     0     0     0]
 [ 9409  2140     7 ...  5347  1292   235]
 [    8     4  7232 ...   168  2352   208]] 

Maximum review length:  130


In [8]:
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           2960128   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2985025 (11.39 MB)
Trainable params: 2985025 (11.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [9]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [10]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 [==============================] - ETA: 0s - loss: 0.4867 - accuracy: 0.7336
Epoch 1: accuracy improved from -inf to 0.73360, saving model to models\LSTM.h5
313/313 [==============================] - 48s 146ms/step - loss: 0.4867 - accuracy: 0.7336
Epoch 2/5


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - ETA: 0s - loss: 0.2219 - accuracy: 0.9206
Epoch 2: accuracy improved from 0.73360 to 0.92058, saving model to models\LSTM.h5
313/313 [==============================] - 47s 150ms/step - loss: 0.2219 - accuracy: 0.9206
Epoch 3/5
313/313 [==============================] - ETA: 0s - loss: 0.1269 - accuracy: 0.9605
Epoch 3: accuracy improved from 0.92058 to 0.96050, saving model to models\LSTM.h5
313/313 [==============================] - 50s 159ms/step - loss: 0.1269 - accuracy: 0.9605
Epoch 4/5
313/313 [==============================] - ETA: 0s - loss: 0.0788 - accuracy: 0.9782
Epoch 4: accuracy improved from 0.96050 to 0.97815, saving model to models\LSTM.h5
313/313 [==============================] - 56s 178ms/step - loss: 0.0788 - accuracy: 0.9782
Epoch 5/5
313/313 [==============================] - ETA: 0s - loss: 0.0593 - accuracy: 0.9851
Epoch 5: accuracy improved from 0.97815 to 0.98505, saving model to models\LSTM.h5
313/313 [=============

In [11]:
loaded_model = load_model('models/LSTM.h5')

In [13]:
review = str(input('Movie Review: '))

Movie Review:  nice movie


In [14]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  nice movie
Filtered:  ['nice movie']


In [15]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[236   3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]]


In [16]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 [==============================] - 2s 2s/step
[[0.9358544]]


In [17]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
